# Домашнее задание "Проблема качества данных"
### Задание

- Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. 
- По образу практики, попробуйте создать искусственный датасет с лишними столбцами. 
- Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. 
- Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. 
- Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) 
- Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [318]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

### Создадим искусственный датасет с лишними столбцами и малым количеством данных, чтобы вставить какие-то не такие данные и увидеть зависимость влияния "кривых "  данных на качество модели

In [319]:
n_samples = 10
#кривые данные
ejection = [5, 0, -5, 'No matter', '3'] # есть неверно введённые данные
ejection2 = [3.141593, None, 4.847876e+04, None, 'second'] # есть пропуски в нецелевых переменных
ejection3 = [3.141593, 31.006277, 4.847876e+04, 'white', None] # есть пропуски в целевой переменной

colour_list = ['yellow', 'red', 'black', 'white', 'orange', 'blue']
places_list = ['first', 'second', 'third']

colours = np.random.choice(colour_list, n_samples)
places = np.random.choice(places_list, n_samples)
radius = np.random.choice(5, n_samples) + np.pi
square = np.pi * radius ** 2 

points = square ** radius + np.e 

In [320]:
data = pd.DataFrame({'Radius': radius, 'Square': square, 'Points': points, 'Colours': colours, 'Places': places})
data.head(15)

,Radius,Square,Points,Colours,Places
0,5.141593,83.051065,7.387393e+09,blue,third
1,4.141593,53.887078,1.482874e+07,blue,first
2,6.141593,118.498237,5.443694e+12,orange,first
3,3.141593,31.006277,4.847876e+04,white,second
4,3.141593,31.006277,4.847876e+04,blue,second
5,5.141593,83.051065,7.387393e+09,black,first
6,5.141593,83.051065,7.387393e+09,yellow,second
7,6.141593,118.498237,5.443694e+12,red,first
8,7.141593,160.228594,5.563590e+15,white,second
9,6.141593,118.498237,5.443694e+12,red,first


### Вставим "кривые" строки с данными

In [321]:
data.loc[len(data)] = ejection
data.loc[len(data)] = ejection2
data.loc[len(data)] = ejection3
data

,Radius,Square,Points,Colours,Places
0,5.141593,83.051065,7.387393e+09,blue,third
1,4.141593,53.887078,1.482874e+07,blue,first
2,6.141593,118.498237,5.443694e+12,orange,first
3,3.141593,31.006277,4.847876e+04,white,second
4,3.141593,31.006277,4.847876e+04,blue,second
5,5.141593,83.051065,7.387393e+09,black,first
6,5.141593,83.051065,7.387393e+09,yellow,second
7,6.141593,118.498237,5.443694e+12,red,first
8,7.141593,160.228594,5.563590e+15,white,second
9,6.141593,118.498237,5.443694e+12,red,first


### Посмотрим какие у нас данные по количеству, по типу, по названиям признаков

In [322]:
# Видно, что есть недостаток данных, попробуем и так протестировать, что получится
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Radius   13 non-null     float64
 1   Square   12 non-null     object 
 2   Points   13 non-null     float64
 3   Colours  12 non-null     object 
 4   Places   12 non-null     object 
dtypes: float64(2), object(3)
memory usage: 624.0+ bytes


### Увидели, что есть категориальные переменные, которые надо бы перевести в числа для усваивания линейной модели. Используем LabelEncoder для последних двух столбцов: Colours, Places(целевая переменная)

In [323]:
from sklearn.preprocessing import LabelEncoder

In [324]:
le = LabelEncoder().fit(data['Colours'])

In [325]:
X_colours = pd.Series(data = le.transform(data['Colours']))
X_colours

0     2
1     2
2     3
3     5
4     2
5     1
6     6
7     4
8     5
9     4
10    0
11    7
12    5
dtype: int32

In [326]:
le1 = LabelEncoder().fit(data['Places'])

In [327]:
y = pd.Series(data = le1.transform(data['Places']))
y

0     3
1     1
2     1
3     2
4     2
5     1
6     2
7     1
8     2
9     1
10    0
11    2
12    4
dtype: int32

### Обновим наш датасет с учётом преобразованных переменных

In [328]:
data['Colours_label'] = X_colours
data['Places_label'] = y
data

,Radius,Square,Points,Colours,Places,Colours_label,Places_label
0,5.141593,83.051065,7.387393e+09,blue,third,2,3
1,4.141593,53.887078,1.482874e+07,blue,first,2,1
2,6.141593,118.498237,5.443694e+12,orange,first,3,1
3,3.141593,31.006277,4.847876e+04,white,second,5,2
4,3.141593,31.006277,4.847876e+04,blue,second,2,2
5,5.141593,83.051065,7.387393e+09,black,first,1,1
6,5.141593,83.051065,7.387393e+09,yellow,second,6,2
7,6.141593,118.498237,5.443694e+12,red,first,4,1
8,7.141593,160.228594,5.563590e+15,white,second,5,2
9,6.141593,118.498237,5.443694e+12,red,first,4,1


In [329]:
from sklearn.metrics import mean_absolute_error

### Возьмём для построения модели те признаки, где нет искажений(кроме Colours - там выброс закодировался) и будем исключать признаки, чтобы увидеть тенденцию ошибки

In [330]:
X = data[['Radius', 'Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Radius', 'Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-3.76243958e-01  1.79412033e-16  1.76842392e-01]
Bias: 2.8041771709221757
Error: 0.5175487180969361


In [331]:
X = data[['Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-8.39799211e-18  2.38570517e-01]
Bias: 0.8517397353400783
Error: 0.6615041348689641


In [332]:
X = data[[ 'Points']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Points']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [5.95469164e-17]
Bias: 1.6667485816408383
Error: 0.7694560609529008


### Получается, если не брать в учёт признаки с косячными данными получается ошибка  0,737, 0,74 и 0,77. Самый лучший вариант пока набор признаков 'Radius', 'Points', 'Colours_label'. Попробуем исключить неверные данные.

In [333]:
data = data.drop(data[data.Square == 0].index)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 12
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Radius         12 non-null     float64
 1   Square         11 non-null     object 
 2   Points         12 non-null     float64
 3   Colours        11 non-null     object 
 4   Places         11 non-null     object 
 5   Colours_label  12 non-null     int32  
 6   Places_label   12 non-null     int32  
dtypes: float64(2), int32(2), object(3)
memory usage: 672.0+ bytes


In [334]:
data

,Radius,Square,Points,Colours,Places,Colours_label,Places_label
0,5.141593,83.051065,7.387393e+09,blue,third,2,3
1,4.141593,53.887078,1.482874e+07,blue,first,2,1
2,6.141593,118.498237,5.443694e+12,orange,first,3,1
3,3.141593,31.006277,4.847876e+04,white,second,5,2
4,3.141593,31.006277,4.847876e+04,blue,second,2,2
5,5.141593,83.051065,7.387393e+09,black,first,1,1
6,5.141593,83.051065,7.387393e+09,yellow,second,6,2
7,6.141593,118.498237,5.443694e+12,red,first,4,1
8,7.141593,160.228594,5.563590e+15,white,second,5,2
9,6.141593,118.498237,5.443694e+12,red,first,4,1


In [335]:
X = data[['Radius', 'Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Radius', 'Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-3.92883330e-01  1.94357389e-16  7.87189893e-02]
Bias: 3.330286891950649
Error: 0.5434611944560169


In [336]:
X = data[['Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-2.29168147e-18  1.50837222e-01]
Bias: 1.2561896036732172
Error: 0.6460450786773603


In [337]:
X = data[[ 'Points']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Points']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [3.22176443e-17]
Bias: 1.8183522799818377
Error: 0.6820479645633092


### Можем заметить, что после удаления всего лишь одной строки с ошибочными данными, мы получаем уже следующие результаты 0,66, 0,68. Самый лучший вариант пока набор признаков 'Radius', 'Points', 'Colours_label'. Теперь попробуем убрать все искажения в данных и получить модель на всех признаках, далее проанализируем, что лучше всего сработало.

In [338]:
# заменили "кривые" данные, используемые в расчёте модели, на статистические метрики: мода и медиана
data.iloc[10,1] = data['Square'].median()
data.iloc[10,5] = data['Colours_label'].mode()[0]
data.iloc[11,6] = data['Places_label'].mode()[0]

In [339]:
data

,Radius,Square,Points,Colours,Places,Colours_label,Places_label
0,5.141593,83.051065,7.387393e+09,blue,third,2,3
1,4.141593,53.887078,1.482874e+07,blue,first,2,1
2,6.141593,118.498237,5.443694e+12,orange,first,3,1
3,3.141593,31.006277,4.847876e+04,white,second,5,2
4,3.141593,31.006277,4.847876e+04,blue,second,2,2
5,5.141593,83.051065,7.387393e+09,black,first,1,1
6,5.141593,83.051065,7.387393e+09,yellow,second,6,2
7,6.141593,118.498237,5.443694e+12,red,first,4,1
8,7.141593,160.228594,5.563590e+15,white,second,5,2
9,6.141593,118.498237,5.443694e+12,red,first,4,1


In [340]:
X = data[['Radius', 'Square', 'Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Radius', 'Square', 'Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-2.04279433e-01  1.72631335e-03  1.53956708e-16 -1.70467242e-02]
Bias: 2.4090994669250203
Error: 0.4558190944794611


In [341]:
X = data[['Radius', 'Square', 'Points']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Radius', 'Square', 'Points']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-2.13563411e-01  2.08363014e-03  1.48318857e-16]
Bias: 2.368495285496673
Error: 0.4530111648704569


In [343]:
X = data[['Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[[ 'Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 8.89358696e-17 -2.42236301e-02]
Bias: 1.6247426410048083
Error: 0.5482766585601059


In [345]:
X = data[['Radius', 'Square',  'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['Radius', 'Square',  'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-0.26097485  0.00689732  0.02138007]
Bias: 2.1933122564208043
Error: 0.5209108407409566


In [342]:
X = data[['Square', 'Points', 'Colours_label']]
y = data['Places_label']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[[ 'Square', 'Points', 'Colours_label']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-5.10151028e-03  1.68541919e-16 -3.10163150e-02]
Bias: 2.0338639745810347
Error: 0.49104957897469054


### Таким образом, перебрав достатоочно вариантов, увидели,уже совсем другие результаты, самый лучший вариант ошибки, основанный на следующих признаках: 'Radius', 'Square', 'Points' - 0.4530111648704569